Il dataset analizzato (https://www.kaggle.com/datasets/shuhengmo/uber-nyc-forhire-vehicles-trip-data-2021), contiene informazioni sui viaggi fatti da varie licenze "taxi" nella città di New York.
La grandezza totale del dataset è di 5 GB e contiene molte informazioni sui luogi di partenza, arrivo, prezzo e varie tempistiche come l'orario e giorno di partenza e quelli d'arrivo.



In [ ]:
%%configure -f
{"executorMemory":"5G", "numExecutors":2, "executorCores":3, "conf":
{"spark.dynamicAllocation.enabled": "false"}}


In [1]:
val bucketname = "unibo-bd-antonelli2023"
val path_ml_tripdata =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-01.parquet"
//"s3a://"+bucketname+"/datasets/project/2017/"
val path_ml_tripdata2 =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-02.parquet"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1686229754695_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd-antonelli2023
path_ml_tripdata: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-01.parquet
path_ml_tripdata2: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-02.parquet
res3: String = application_1686229754695_0001
res5: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1686229754695_0001/


I dati sono presenti nel formato Parquet; dopo essere stati importati, tutti i job sono stati fatti dopo una successiva conversione in RDD. I dati sono raggruppati mensilmente, per un totale di 12 file Parquet. 

Importazione dei dati in formato Parquet.

In [122]:
import spark.implicits._
val parquetFileDF = spark.read.parquet(path_ml_tripdata)
val parquetFileDF2 = spark.read.parquet(path_ml_tripdata2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
parquetFileDF: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]
parquetFileDF2: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]


# Analisi dei campi del dataset
* licenseClass: tipo stringa che rappresenta la licenza de taxi.
	* HV0002: Juno
    * HV0003: Uber
    * HV0004: Via
    * HV0005: Lyft
* license: tipo Stringa che rappresenta il numero del taxi.
* request: tipo Timestamp che rappresenta la data/ora della richiesta del viaggio.
* pickup: tipo Timestamp  che rappresenta la data/ora dell'inizio del viaggio.
* dropoff: tipo Timestamp che rappresenta la data/ora della fine del viaggio.
* distance: tipo Double che rappresenta la distanza totale in miglia percorsa dal taxi per questo viaggio.
* startloc: tipo Long che rappresenta la zona di partenza del viaggio.
* endloc: tipo Long che rappresenta la zona di arrivo del viaggio.
* fare: tipo Double che rappresenta il prezzo del viaggio.

In [123]:
import java.sql.Timestamp
case class TaxiTrip(
  licenseClass:String,
  license:String,  
  request:Timestamp,
  pickup:Timestamp,
  dropoff:Timestamp,
  distance:Double,
  startloc:Long,
  endloc:Long,
  //time:Long,
  fare:Double,
)

object TaxiTrip {
    def extract(row:org.apache.spark.sql.Row) = {
        val licenseClass = row.getString(0)
        val license = row.getString(1)
        val request = row.getTimestamp(4)
        val pickup = row.getTimestamp(5)
        val dropoff = row.getTimestamp(6)
        val distance = row.getDouble(9)
        val startloc = row.getLong(7)
        val endloc = row.getLong(8)
        //val time = row.getLong(10)
        val fare = row.getDouble(11)
        
        new TaxiTrip(licenseClass,license,request,pickup,dropoff,distance,startloc,endloc,fare)
        

        //new TaxiTrip(license,request,pickup,dropoff,startloc,endloc,distance,time,fare)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.sql.Timestamp
defined class TaxiTrip
defined object TaxiTrip
Companions must be defined together; you may wish to use :paste mode for this.


La seguente funzione serve per convertire da formato timestamp a calendar, in modo da poter estrarre informazioni come giorno del mese o ora del giorno. <br>
Si può notare che il dataset conteneva "trip_time" ossia il totale del tempo in secondi che i passeggeri hanno passato sul taxi durante il viaggio. <br>Ho preferito definire una funzione perchè sarebbe stata utile per tutti i tempi importati dal dataset (request, pickup e dropoff).

In [124]:
import java.util.Calendar
def getDayTime(time: Long): Calendar = {
    var date:Calendar = Calendar.getInstance();
    date.setTimeInMillis(time);
    date
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.util.Calendar
getDayTime: (time: Long)java.util.Calendar


# Creazione e caching del RDD a partire dal file Parquet.
Caching per migliorare le prestazioni in vista di futuri utilizzi. Si è optato per la scelta tenere due ripartizioni (quindi due task per core). Essendoci due executor con tre core ciascuno, il numero totale di core è sei. Quindi si è scelta una ripartizione di 12, anche se guardando i tempi di esecuzione il miglioramento è solo di qualche secondo.

In [125]:
import org.apache.spark.HashPartitioner
val p = new HashPartitioner(12)
val data_rdd = parquetFileDF.rdd
val data_rdd2 = parquetFileDF2.rdd
val rddTaxiTrip = data_rdd.map(TaxiTrip.extract)
val rddTaxiTrip2 = data_rdd2.map(TaxiTrip.extract)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.HashPartitioner
p: org.apache.spark.HashPartitioner = org.apache.spark.HashPartitioner@c
data_rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[348] at rdd at <console>:56
data_rdd2: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[354] at rdd at <console>:56
rddTaxiTrip: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[355] at map at <console>:57
rddTaxiTrip2: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[356] at map at <console>:57


# File di input
Per tutte le seguenti query verranno utilizzati due file, i quali verranno uniti successivamente attraverso una union: il primo contenente i dati per il mese di gennaio 2021, il secondo contente i dati di febbraio 2021.
<br>Avendo caricato tutti i file nella cartella è possibile fare confronti anche su mesi diversi, ed avendo Kaggle anche i dataset di anni precedenti la stessa analisi può essere effettutata anche su anni differenti. (Attenzione ad eventuali campi mancanti in dataset di anni precedenti). 
<br>La grandezza di input è di circa 380/390 MB a file.


## Numero totale di record

In [ ]:
rddTaxiTrip.count()

## Numero totale di zone di arrivo diverse
Lo stesso numero sono quelle di partenza.

In [ ]:
rddTaxiTrip.map(x => x.endloc).distinct().count()

In [109]:
//rddTaxiTrip.map(x => x.startloc).distinct().count()
//rddTaxiTrip.map(x => x.license).distinct().count()
//rddTaxiTrip2.map(x => x.startloc).distinct().count()
//rddTaxiTrip2.map(x => x.license).distinct().count()
//rddTaxiTrip.map(x => x.startloc).take(5)
//rddTaxiTrip.map(x => x.license).take(5)
rddTaxiTrip.map(x => x.request).take(5)
rddTaxiTrip.map(x => x.request.getTime()).take(5)
//rddTaxiTrip.map(x => x.fare).take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res193: Array[java.sql.Timestamp] = Array(2021-01-01 00:31:42.0, 2021-01-01 00:55:19.0, 2021-01-01 00:22:41.0, 2021-01-01 00:42:37.0, 2021-01-01 00:47:17.0)
res194: Array[Long] = Array(1609461102000, 1609462519000, 1609460561000, 1609461757000, 1609462037000)


## Stampa delle classi di licenza 
Nel caso ne mancasse qualcuna significa che non è presente nel periodo di tempo considerato.

In [ ]:
rddTaxiTrip.map(x => x.licenseClass).distinct().collect()

### VIAGGI PIU LUNGHI IN TERMINE DI KM, IN RELAZIONE CON LA DURATA DEL VIAGGIO


In [ ]:
val query1 = rddTaxiTrip.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))
val query1802 = rddTaxiTrip2.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))

In [ ]:
val query1union = query1.union(query1802).cache()

In [ ]:
query1union.sortByKey(false).take(10)

### VIAGGI PIU COSTOSI, IN RELAZIONE CON LA SOMMA TRA LUNGHEZZA E LA DURATA DEL VIAGGIO
Aggiunto un filtro sulla distanza minima. Questo comporterà solo la stampa dei viaggi più costosi visto che ci si aspetta che l'andamento sia proporzionale tra distanza e costo.

In [ ]:
val query2 = rddTaxiTrip.filter(_.distance > 10).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000) + x.distance)))
val query2802 = rddTaxiTrip2.filter(_.distance > 10).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000) + x.distance)))

In [ ]:
val query2union = query2.union(query2802).cache()

In [ ]:
query2union.sortByKey(false).take(10)

### TEMPO DI VIAGGIO MEDIO, IN BASE ALLA ZONA DI PARTENZA

In [110]:
val query5 = rddTaxiTrip.map(x => (x.startloc, (x.dropoff.getTime() - x.request.getTime())/(60*1000))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)
val query51 = rddTaxiTrip2.map(x => (x.startloc, (x.dropoff.getTime() - x.request.getTime())/(60*1000))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)
val query5union = query5.union(query51).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query5: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[305] at partitionBy at <console>:52
query51: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[308] at partitionBy at <console>:52
query5union: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = PartitionerAwareUnionRDD[309] at union at <console>:51


##### Risultato raggruppato per vedere la differenza tra i due mesi rispetto alla zona di partenza

In [111]:
query5union.map({case(k,v) => (k,v._1/v._2)}).groupByKey().map({case(k,v) => (v,k)}).sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 256.0 failed 4 times, most recent failure: Lost task 1.3 in stage 256.0 (TID 894) (ip-172-31-16-174.ec2.internal executor 1): java.lang.NullPointerException
	at $anonfun$query5$1(<console>:51)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:199)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:65)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:52)
	at org.apache.spark.scheduler.Task.run(Task.scala:138)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:548)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1516)
	a

#### Attenzione

In [68]:
//CAPIRE QUALE è IL RISULTATO
val query6 = rddTaxiTrip.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))
val query61 = rddTaxiTrip2.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))
val query6union = query5.union(query51).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query6: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[203] at map at <console>:30
query61: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[204] at map at <console>:29
query6union: org.apache.spark.rdd.RDD[(Long, Long)] = UnionRDD[205] at union at <console>:30


In [82]:
//query5union.take(3)
query6union.aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res153: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[242] at partitionBy at <console>:32


In [86]:

query6union.map({case(k,v) => (k/v)}).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res157: Array[Long] = Array(15, 6, 16, 71, 7, 2, 3, 2, 16, 19)


In [83]:
query6union.map({case(k,v) => (k,v._1/v._2)}).sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:30: error: value _1 is not a member of Long
       query6union.map({case(k,v) => (k,v._1/v._2)}).sortByKey(false).take(10)
                                          ^
<console>:30: error: value _2 is not a member of Long
       query6union.map({case(k,v) => (k,v._1/v._2)}).sortByKey(false).take(10)
                                               ^



IN MEDIA QUANTI VIAGGI FA CIASCUNA LICENZA

In [112]:
val q = rddTaxiTrip.map(x => (x.license,1)).
reduceByKey(_+_).
aggregate((0,0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
"Media: " + (q._1/q._2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

q: (Int, Int) = (11908468,32)
res198: String = Media: 372139


CLASSE DI LICENZA CHE FA PIU VIAGGI, CON LA PRESENZA ANCHE DELLA TARIFFA CLIENTE

In [113]:
rddTaxiTrip.map(x => (x.licenseClass,(x.fare,1))).
reduceByKey((t1,t2) => (t1._1+t2._1, t1._2+t2._2)).
map(v => (v._1,v._2._1, v._2._2)).
sortBy(_._2,false).
collect().foreach(println(_))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(HV0003,1.468278860287933E8,8704128)
(HV0005,5.509768345985989E7,3094325)
(HV0004,2816007.860000914,110015)


LICENZA CHE FA PIU VIAGGI, CON LA PRESENZA ANCHE DELLA TARIFFA CLIENTE

In [114]:
rddTaxiTrip.map(x => (x.license,(x.fare,1))).
reduceByKey((t1,t2) => (t1._1+t2._1, t1._2+t2._2)).
map(v => (v._1,v._2._1, v._2._2)).
sortBy(_._2,false).
collect().foreach(println(_))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(B02510,5.503748608986006E7,3091000)
(B02764,1.709378720999917E7,1009388)
(B02872,1.5509604710001156E7,924960)
(B02875,1.2014741050004447E7,735450)
(B02765,1.0189792220006326E7,591242)
(B02869,7744772.440006703,452098)
(B02887,5608155.9400040135,333768)
(B02871,5581241.000004057,330085)
(B02682,5456731.730003582,321599)
(B02866,5284853.590003659,309274)
(B02864,5203100.810003353,316395)
(B02878,5125333.480003258,312013)
(B02617,4884453.930003084,281432)
(B02883,4582633.870002626,268391)
(B02884,4354322.950002186,257674)
(B02882,4097472.360001952,241988)
(B02876,3929723.280001779,230732)
(B02867,3730036.2400015946,217449)
(B02879,3572925.7100014733,216993)
(B02877,3490842.3500013095,208986)
(B02835,3355979.0300012426,200129)
(B02888,3032522.3300008504,177542)
(B02800,2776613.2300008354,108146)
(B02889,2500286.6500002183,149398)
(B02836,2385524.3700000723,140365)
(B02395,2056890.0899998054,124107)
(B02880,2039189.9299998167,119173)
(B02870,1768859.4399998628,106975)
(B02865,1385092.24999

MAP PER QUERY GROUP BY LOCATION/TIME, REDUCE BY KEY PER CONTARE LE OCCORRENZE DEI RECORD, 
MAP PER SCAMBIARE CHIAVE-VALORE PER POTER ORDINARE I VALORI PER VEDERE LE ZONE/PERIODI PIU GETTONATI


In [ ]:
rddTaxiTrip.map(x => (x.startloc, 1)).reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false).collect()

val query4 = rddTaxiTrip.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), 1)).partitionBy(p)
val query4802 = rddTaxiTrip2.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), 1)).partitionBy(p)

val query4union = query4.union(query4802).cache()

query4union.reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false).collect()

In [ ]:
val tableau = rddTaxiTrip.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.DAY_OF_WEEK), 1)).
                    reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false)

DISTANZA MEDIA PERCORSA DAI TAXI IN UN SINGOLO VIAGGIO, IN RELAZIONE ALLA ZONA --> vale come query finale

In [157]:
val query5 = rddTaxiTrip.filter(_.distance < 100).map(x => (x.startloc,x.distance)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)
val query51 = rddTaxiTrip2.filter(_.distance < 100).map(x => (x.startloc,x.distance)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query5: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[382] at partitionBy at <console>:58
query51: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[386] at partitionBy at <console>:58


In [158]:
val query5union = query5.union(query51).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query5union: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = PartitionerAwareUnionRDD[387] at union at <console>:57


In [159]:
query5union.map({case(k,v) => (k,v._1/v._2)}).map({case(k,v) => (v,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res239: Array[(Double, Long)] = Array((20.136333333333326,1), (19.978372093023246,1), (15.583862644195618,132), (14.864652072083206,132), (11.889173913043477,2), (10.915017273061434,138), (10.765875894988065,27), (10.74933302682418,138), (10.515,199), (10.507823529411766,110), (10.40900593471811,27), (9.162642857142858,110), (9.14,199), (8.924624093697727,46), (8.61814971482889,46), (7.930163748712669,44), (7.793376006573554,201), (7.706209863588672,265), (7.667520547945171,201), (7.6189385377943255,44), (7.448386711684551,101), (7.406741350906092,204), (7.399251434034412,194), (7.375441467436126,204), (7.307814302191469,265), (7.277923984132074,101), (7.179391827697307,117), (7.034868680734756,15), (6.987986581402682,19), (6.9764677950164495,194), (6.974829246935218,117), (6.9686778535...


SQL

In [160]:
parquetFileDF.createOrReplaceTempView("parquetFile")
val parkDF = spark.sql("Select * from parquetFile")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

parkDF: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]


In [161]:
parkDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res241: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]


ANALISI SUL PREZZO MEDIO DI OGNI SINGOLO VIAGGIO PER OGNI GIORNO

In [139]:
val avgPricesByYear = rddTaxiTrip.map(x => ((x.licenseClass, getDayTime(x.pickup.getTime()).get(Calendar.DAY_OF_WEEK)), (x.fare, 1))).
  reduceByKey((accum, value) => (accum._1 + value._1, accum._2 + value._2)).
  mapValues(sumCount => sumCount._1 / sumCount._2).
  map(item => (item._1._1, item._2, item._1._2))
  //collect() //(licenza,avg,giorno)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgPricesByYear: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[370] at map at <console>:60


In [140]:
// Raggruppo per licenza
val avgPricesByProduct = avgPricesByYear.groupBy(_._1).
  mapValues(_.map(item => (item._2, item._3)))//.collect()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgPricesByProduct: org.apache.spark.rdd.RDD[(String, Iterable[(Double, Int)])] = MapPartitionsRDD[373] at mapValues at <console>:58


In [155]:
//flatmap con chiave prodotto e anno
val avgPricesByProductAndYear = avgPricesByProduct.flatMap { 
    case (itemName, yearPrices) => yearPrices.map { 
        case (avgPrice, year) => ((itemName, year),avgPrice)
        //(itemName,(year , avgPrice))
  }
}.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgPricesByProductAndYear: Array[((String, Int), Double)] = Array(((HV0003,7),16.623800537801152), ((HV0003,3),16.708234621891965), ((HV0003,5),17.02267218913682), ((HV0003,6),17.248924009839303), ((HV0003,1),16.817135915591873), ((HV0003,4),16.717073180314657), ((HV0003,2),16.85329002257803), ((HV0004,7),26.234603123836248), ((HV0004,3),24.799659412100148), ((HV0004,6),26.32422286481737))


In [149]:
avgPricesByProductAndYear.groupByKey().take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:57: error: value groupByKey is not a member of Array[(String, (Int, Double))]
       avgPricesByProductAndYear.groupByKey().take(10)
                                 ^



In [156]:


// Conversione in DataFrame con stringa
val df = avgPricesByProductAndYear.map{case ((itemName, year), avgPrice) => (itemName,year,avgPrice)}.toDF()

// Salvataggio su file CSV
df.write.format("csv").mode(SaveMode.Overwrite).save(path_itemsForYear_revenue)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:57: error: value toDF is not a member of Array[(String, Int, Double)]
       val df = avgPricesByProductAndYear.map{case ((itemName, year), avgPrice) => (itemName,year,avgPrice)}.toDF()
                                                                                                             ^

